Question: Can we predict the amount of calories burnt during a workout based on biological measures?

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib as plt


### Dataset Creation and Initial Treatment

In [ ]:
# loading
df = pd.read_csv("calories.csv") # source: https://www.kaggle.com/datasets/ruchikakumbhar/calories-burnt-prediction/data

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
# setting up and cleaning
df = df.set_index('User_ID')  # set the column 'User_ID' as the dataset index
df = df.drop_duplicates()  # remove duplicates
df = df.fillna(0)  # fill empty columns with 0

### EDA (Exploratory Data Analisys)